In [1]:
import psycopg2

In [2]:
def deleteResultsMethods():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")

        cur.execute("""DELETE FROM actions a
                       USING feature_actions fa,
                             results r,
                             dataset_results dr,
                             datasets ds,
                             sampling_features sf 
                       WHERE a.actionid=fa.actionid
                         AND fa.featureactionid=r.featureactionid
                         AND r.resultid=dr.resultid
                         AND dr.datasetid=ds.datasetid
                         AND fa.samplingfeatureid=sf.samplingfeatureid
                         AND ds.datasetcode LIKE 'T1Data_%'""")

        cur.execute("""DELETE FROM feature_actions fa
                       USING results r,
                             dataset_results dr,
                             datasets ds,
                             sampling_features sf
                       WHERE fa.featureactionid=r.featureactionid
                         AND r.resultid=dr.resultid
                         AND dr.datasetid=ds.datasetid    
                         AND fa.samplingfeatureid=sf.samplingfeatureid
                         AND ds.datasetcode LIKE 'T1Data_%'""")

        cur.execute("""DELETE FROM measurement_result_values rv 
                       USING results r,
                             dataset_results dr,
                             datasets ds 
                       WHERE rv.resultid=r.resultid
                         AND r.resultid=dr.resultid
                         AND dr.datasetid=ds.datasetid
                         AND ds.datasetcode LIKE 'T1Data_%'""")

        cur.execute("""DELETE FROM results r
                       USING dataset_results dr,
                             datasets ds 
                       WHERE r.resultid=dr.resultid
                         AND dr.datasetid=ds.datasetid
                         AND ds.datasetcode LIKE 'T1Data_%'""") 

        cur.execute("""DELETE FROM dataset_results dr
                       USING datasets ds 
                       WHERE dr.datasetid=ds.datasetid
                         AND ds.datasetcode LIKE 'T1Data_%'""") 

        cur.execute("""DELETE FROM datasets ds
                       WHERE ds.datasetcode LIKE 'T1Data_%'""") 

        conn.commit()
        cur.execute("""ANALYZE actions""")
        cur.execute("""ANALYZE feature_actions""")
        cur.execute("""ANALYZE measurement_result_values""")
        cur.execute("""ANALYZE results""")
        cur.execute("""ANALYZE dataset_results""")
        cur.execute("""ANALYZE datasets""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close() 
            

In [3]:
deleteResultsMethods()